# Holiday Destination Recommender - Report

## 1. Introduction

I am sure you know this problem very well.

You are so ready to go onto your next vacation - only you do not know what destination you should choose. Not so much because you have too much choice and you cannot decide - the issue is rather, you do not actually know what are the destinations that really match your current *mood* for holiday destinations? And is there anything exotic to discover other than all the mainstream places?

So here is the idea.

A *Holiday Destination Recommender*: You tell the recommender the places you like and that match your current mood & the recommender shows you similar places worth visting - places you might have never come across yourself.
Yes, very similar to what you already know from Netflix, Spotify and YouTube.

The project is implemented using an unsupervised learning approach and hence does not depend on the availability of labeled training data. Please refer to section **Methodology** for more details on the approach.



## 2. Data

In this section, I present which type of data is needed, where they are retrieved from and how they are processed to implement the Holiday Destination Recommender.

## 2.1 Data Requirements

To implement the Holiday Destination Recommender, the following data is required:
- **A list of cities that serve as "benchmark cities"**; in this example, we go with the cities "Barcelona, Spain" and "Budapest, Hungary"
- **A list of all cities in the world**: This is needed, so that the recommender can scout new cities to suggest to the user.
- **The latitudes and longitudes of those cities**: This is needed, so that places are uniquely identified and more information can be retrieved about them.
- **The available venues of those cities**: This is needed, so that the recommender can calculate the similarity between the benchmark cities and possible holiday destinations. First and foremost, the available *venue categories* are of interest, since they form the basis of the comparison.

## 2.2 Data Sources

The following data sources are used to retrieve above mentioned data:
- Dataset from https://datahub.io/core/world-cities: Here a CSV is provided that contains all major cities of the world with more than 150000 inhabitants. In total there are more than 20k entries.
- Python library *geopy* to request *Nominatim* interface for retrieving the coordinates of cities.
- Foursquare Places API to obtain information on the available venues for a city.

In the following you can find some extracts of the retrieved data:

1) Benchmark cities with coordinates and available venues

![title](img/Benchmark_Cities.png)

2) Destination cities (i.e. possible suggestions by the recommender) with coordinates and available venues

![title](img/New_Cities.png)